In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import re # Import the regular expression module

# --- Configuration ---
FILE_PATH = r"C:\Users\acer\Downloads\Enterprise_Sustainable Power Evaluation_Dataset.csv"
SEQUENCE_LENGTH = 10  # Number of historical time steps used to predict the next step

# Define the target variable (Emissions Intensity is a key EPM metric)
TARGET_COLUMN = 'Emissions Intensity (kg CO₂ per MWh)'
# Features to use for prediction (other key EPM metrics)
FEATURE_COLUMNS = [
    'Revenue (USD)',
    'Net Profit Margin (%)',
    'Energy Efficiency (%)',
    'Renewable Energy Share (%)',
    'Sustainability Score',
    'Innovation Index'
]

def clean_column_name(col_name):
    """Helper function to clean column names using regex substitution."""
    # 1. Replace non-alphanumeric characters (except space) with '_'
    cleaned = re.sub(r'[^A-Za-z0-9%]+', '_', col_name)
    # 2. Strip leading/trailing underscores
    cleaned = cleaned.strip('_')
    # 3. Remove parentheses and percentage signs
    cleaned = re.sub(r'[%()]', '', cleaned)
    return cleaned.replace(' ', '_')

def load_and_preprocess_data(file_path):
    """Loads, cleans, and structures the cross-sectional data into a time series."""
    print("--- 1. Loading and Cleaning Data ---")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Please ensure the file is in the correct directory.")
        return None, None

    # Standardize column names in the DataFrame
    df.columns = df.columns.str.replace('[^A-Za-z0-9%]+', '_', regex=True).str.strip('_')
    df.columns = df.columns.str.replace('[%()]', '', regex=True)
    df.columns = df.columns.str.replace(' ', '_')

    # Convert the dataset into a single pseudo-time-series
    # We sort by Company_ID just to ensure the sequence is stable across runs,
    # treating each row as a sequential time step for our "Composite Enterprise".
    df_ts = df.sort_values(by='Company_ID').reset_index(drop=True)

    # Apply the custom clean_column_name function to the configuration list
    cleaned_feature_cols = [clean_column_name(col) for col in FEATURE_COLUMNS]
    cleaned_target_col = clean_column_name(TARGET_COLUMN)
    
    cols_to_use = cleaned_feature_cols + [cleaned_target_col]

    # Select only the features and the target, and drop any rows with NaN values
    # Note: We must use the cleaned DataFrame columns here
    df_ts = df_ts[cols_to_use].dropna()

    print(f"Dataset size after cleaning: {df_ts.shape}")
    return df_ts, df_ts.columns

def create_sequences(data, sequence_length):
    """
    Creates sequences of features (X) and the corresponding target value (Y)
    for use in an LSTM model.
    X: [t-N, t-N+1, ..., t-1] (Sequence of historical features)
    Y: [t] (Target value at the next time step)
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        # Extract features (all columns except the last one, which is the target)
        X.append(data[i:i + sequence_length, :-1])
        # Extract the target value at the next step (i + sequence_length)
        # Note: We are predicting the target for the step immediately *after* the sequence ends.
        y.append(data[i + sequence_length, -1])
    return np.array(X), np.array(y)

def build_and_train_baseline(df_ts, feature_names):
    """Scales data, creates sequences, and trains the LSTM baseline model."""
    print("\n--- 2. Data Scaling and Sequence Creation ---")

    # The last column is the target (Emissions_Intensity_kg_CO2_per_MWh)
    data = df_ts.values
    
    # Initialize separate scalers for features and target
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()

    # Scale the features (all columns except the last one)
    features_scaled = scaler_features.fit_transform(data[:, :-1])
    
    # Scale the target separately (the last column)
    target_scaled = scaler_target.fit_transform(data[:, -1].reshape(-1, 1))

    # Recombine scaled data for sequence creation
    scaled_data = np.hstack((features_scaled, target_scaled))
    
    X, y = create_sequences(scaled_data, SEQUENCE_LENGTH)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False, random_state=42
    )

    print(f"X_train shape: {X_train.shape} (Samples, Time Steps, Features)")
    print(f"y_train shape: {y_train.shape}")
    
    # --- 3. Build LSTM Model (Baseline) ---
    print("\n--- 3. Building and Training LSTM Baseline ---")
    
    # Define the required input shape for the LSTM layers
    input_seq_shape = (X_train.shape[1], X_train.shape[2])

    # Using tf.keras.Input as the first layer to explicitly define shape, 
    # which resolves the UserWarning when defining Sequential models.
    model = Sequential([
        tf.keras.Input(shape=input_seq_shape),
        LSTM(units=50, return_sequences=True),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False),
        Dropout(0.2),
        Dense(units=1)  # Output layer for the single target variable
    ], name="EPM_LSTM_Baseline")
    
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        verbose=1,
        shuffle=False # Important for time-series data
    )
    
    # --- 4. Evaluate and Inverse Transform ---
    print("\n--- 4. Evaluating Baseline Performance ---")
    
    # Predict on the test set
    y_pred_scaled = model.predict(X_test)
    
    # Inverse transform to get the prediction in the original scale (kg CO2 per MWh)
    y_test_original = scaler_target.inverse_transform(y_test.reshape(-1, 1))
    y_pred_original = scaler_target.inverse_transform(y_pred_scaled)
    
    # Calculate Mean Absolute Error (MAE) - a common time-series metric
    mae = np.mean(np.abs(y_pred_original - y_test_original))
    
    print(f"\nBaseline Model Evaluation:")
    print(f"Target Variable: {TARGET_COLUMN}")
    print(f"Sequence Length (Lookback): {SEQUENCE_LENGTH} steps")
    print(f"Test MAE (Emissions Intensity): {mae:.2f} kg CO₂ per MWh")
    print(f"This MAE value serves as the benchmark for the Hybrid Model (Phase 3).")


if __name__ == "__main__":
    df_ts, feature_names = load_and_preprocess_data(FILE_PATH)
    if df_ts is not None:
        build_and_train_baseline(df_ts, feature_names)


--- 1. Loading and Cleaning Data ---
Dataset size after cleaning: (1000, 7)

--- 2. Data Scaling and Sequence Creation ---
X_train shape: (792, 10, 6) (Samples, Time Steps, Features)
y_train shape: (792,)

--- 3. Building and Training LSTM Baseline ---


Model: "EPM_LSTM_Baseline"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 50)              │          11,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,651 (123.64 KB)

 Trainable params: 31,651 (123.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - loss: 0.1360 - val_loss: 0.0877
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0901 - val_loss: 0.0776
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0861 - val_loss: 0.0778
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0886 - val_loss: 0.0775
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0866 - val_loss: 0.0774
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0869 - val_loss: 0.0773
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0855 - val_loss: 0.0773
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0861 - val_loss: 0.0773
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0862 - val_loss: 0.0773
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0873 - val_loss: 0.0774
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0858 - val_loss: 0.0772
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
import random

# --- Configuration (matching Phase 1) ---
FILE_PATH = r"C:\Users\acer\Downloads\Enterprise_Sustainable Power Evaluation_Dataset.csv"
EMBEDDING_DIMENSION = 384  # Standard dimension for many sentence transformers (e.g., MiniLM)

# Define the target variable (Emissions Intensity is a key EPM metric)
TARGET_COLUMN = 'Emissions Intensity (kg CO₂ per MWh)'
# Ensure this constant exactly matches the result of cleaning the TARGET_COLUMN
TARGET_CLEANED = 'Emissions_Intensity_kg_CO2_per_MWh'

# Features to use for prediction (other key EPM metrics)
FEATURE_COLUMNS = [
    'Revenue (USD)',
    'Net Profit Margin (%)',
    'Energy Efficiency (%)',
    'Renewable Energy Share (%)',
    'Sustainability Score',
    'Innovation Index'
]

def load_and_clean_data(file_path):
    """Loads and cleans the initial CSV, standardizing column names."""
    print("--- 1. Loading and Cleaning Data ---")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found. Please ensure the file is accessible.")
        return None

    # --- Robust Column Cleaning Fix ---
    def standardize_column_name(col_name):
        # 1. Start cleaning
        cleaned = col_name
        
        # 2. Aggressively clean up known problematic strings (like CO₂ encoding issues)
        cleaned = cleaned.replace('â\x82\x82', '2') # Handles one common encoding for CO₂
        
        # 3. Replace all non-alphanumeric, non-space characters with an underscore
        cleaned = re.sub(r'[^A-Za-z0-9\s_]', '', cleaned)
        
        # 4. Replace spaces with underscores
        cleaned = cleaned.replace(' ', '_')
        
        # 5. Replace multiple underscores with a single underscore
        cleaned = re.sub(r'_+', '_', cleaned)
        
        # 6. Strip any leading/trailing underscores
        cleaned = cleaned.strip('_')
        
        # FIX: Explicitly handle the target column's known problematic output 
        # to guarantee the required name, as general regex cleaning is failing.
        if 'Emissions_Intensity_kg_CO_per_MWh' in cleaned:
            return TARGET_CLEANED
        
        return cleaned

    # Apply the standardization to all columns
    df.columns = [standardize_column_name(col) for col in df.columns]

    # Convert the dataset into a single pseudo-time-series
    df_ts = df.sort_values(by='Company_ID').reset_index(drop=True)
    
    # Generate the list of *correctly* clean feature columns using the same function
    # Note: We now have to use the cleaned column names from the df.columns directly 
    # since the source string TARGET_COLUMN is unreliable.
    # We will use the columns that are necessary.
    
    # List of all clean features and targets needed
    required_cols = ['Company_ID', 'Revenue_USD', 'Net_Profit_Margin', 'Energy_Efficiency', 
                     'Renewable_Energy_Share', 'Sustainability_Score', 'Innovation_Index', 
                     TARGET_CLEANED]
    
    # Filter for the required columns
    df_ts = df_ts[required_cols].dropna()
    
    # Validation check: Ensure the critical columns exist now
    if TARGET_CLEANED not in df_ts.columns:
        print(f"\nFATAL ERROR: Target column '{TARGET_CLEANED}' is still missing after cleaning.")
        print(f"Available columns: {df_ts.columns.tolist()}")
        return None

    # Sanity Check for the other columns used later in the script
    if 'Energy_Efficiency' not in df_ts.columns or 'Innovation_Index' not in df_ts.columns:
        print("\nFATAL ERROR: Energy_Efficiency or Innovation_Index is missing after cleaning.")
        return None


    print(f"Dataset size after cleaning: {df_ts.shape}")

    return df_ts

def simulate_narratives_and_embeddings(df):
    """
    Simulates the LLM's role: generating contextual narratives and their embeddings.
    Since we cannot run a live LLM, we use rule-based simulation.
    """
    print("--- 2. Simulating Narrative Context and Embeddings ---")
    
    # 1. Calculate step-wise changes in key indicators (simulating a time-series perspective)
    # The columns here must match the names created in load_and_clean_data
    df['Emissions_Change'] = df[TARGET_CLEANED].diff().fillna(0)
    df['Efficiency_Change'] = df['Energy_Efficiency'].diff().fillna(0)
    df['Innovation_Change'] = df['Innovation_Index'].diff().fillna(0)
    
    narratives = []
    
    for i in range(len(df)):
        emissions_change = df.loc[i, 'Emissions_Change']
        efficiency_change = df.loc[i, 'Efficiency_Change']
        innovation_change = df.loc[i, 'Innovation_Change']
        
        narrative = ""

        # --- Rule-Based Narrative Generation ---
        
        # A. Significant Emissions Improvement (Negative change is good)
        if emissions_change < -50:
            narrative = "Following the deployment of a new carbon capture pilot program, the intensity of emissions saw a strong, unexpected decrease. Management expects this trend to stabilize next quarter, pending full-scale operational review."
        # B. Emissions Spike (Positive change is bad)
        elif emissions_change > 50:
            narrative = "Operational downtime at the primary renewable facility forced a temporary reliance on legacy assets, causing a sharp, but predicted, spike in emissions intensity. This is a short-term impact only."
        # C. Efficiency Drop
        elif efficiency_change < -5:
            narrative = "Initial reports indicate supply chain disruptions affecting key machinery maintenance, leading to a temporary decline in reported energy efficiency. Remedial efforts are underway."
        # D. Innovation/Future Investment
        elif innovation_change > 10:
            narrative = "Significant capital was allocated towards future-proofing and R&D for grid optimization, signalling a forward-looking strategy that may impact short-term profit margins but promises substantial long-term gains in sustainability."
        # E. Baseline/Steady State
        else:
            narrative = "Quarterly review shows stable performance across core metrics with no material changes to operational forecasts. The strategic focus remains on incremental improvements in resource allocation efficiency."

        narratives.append(narrative)

    df['Narrative'] = narratives
    
    # --- 3. Simulated Embedding Generation (Performance Fix) ---
    # Here, we simulate the embeddings using random vectors for simplicity
    print(f"Simulating {len(df)} embeddings of dimension {EMBEDDING_DIMENSION}.")
    np.random.seed(42)
    
    # Create a dummy array of embeddings (0 to 1, consistent with normalized LLM embeddings)
    embeddings = np.random.rand(len(df), EMBEDDING_DIMENSION).astype(np.float32)
    
    # FIX: Create a DataFrame from the embeddings array and concatenate it with the main DataFrame
    # This replaces the slow iterative column insertion and avoids the PerformanceWarning.
    embedding_cols = [f'Embedding_{j}' for j in range(EMBEDDING_DIMENSION)]
    embeddings_df = pd.DataFrame(embeddings, columns=embedding_cols, index=df.index)
    
    # Join the embeddings back to the main DataFrame efficiently
    df = pd.concat([df, embeddings_df], axis=1)
        
    # Drop intermediate change columns
    df = df.drop(columns=['Emissions_Change', 'Efficiency_Change', 'Innovation_Change'])
    
    return df

def save_augmented_data(df):
    """Saves the final DataFrame containing structured data, narrative, and embeddings."""
    AUGMENTED_FILE = 'epm_augmented_data_with_embeddings.csv'
    df.to_csv(AUGMENTED_FILE, index=False)
    print(f"\n--- 3. Data Augmentation Complete ---")
    print(f"Saved augmented dataset to: {AUGMENTED_FILE}")
    print(f"Final shape: {df.shape} (Includes {EMBEDDING_DIMENSION} embedding columns)")
    print("This file is now ready for the Hybrid Model (Phase 3).")
    
if __name__ == "__main__":
    
    # The actual path to the accessible file is the string defined in FILE_PATH, 
    # not the temporary C:\Users path, which is only used for tracking the original upload.
    # We must pass the platform-accessible filename here.
    df_ts_cleaned = load_and_clean_data(FILE_PATH) 
    if df_ts_cleaned is not None:
        df_augmented = simulate_narratives_and_embeddings(df_ts_cleaned)
        save_augmented_data(df_augmented)


--- 1. Loading and Cleaning Data ---
Dataset size after cleaning: (1000, 8)
--- 2. Simulating Narrative Context and Embeddings ---
Simulating 1000 embeddings of dimension 384.

--- 3. Data Augmentation Complete ---
Saved augmented dataset to: epm_augmented_data_with_embeddings.csv
Final shape: (1000, 393) (Includes 384 embedding columns)
This file is now ready for the Hybrid Model (Phase 3).


In [3]:
import pandas as pd
import numpy as np
import re
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam

# --- Configuration (MUST match Phases 1 & 2) ---
FILE_PATH = r"C:\Users\acer\Downloads\Enterprise_Sustainable Power Evaluation_Dataset.csv"
SEQUENCE_LENGTH = 10        # Historical steps for Time-Series input
EMBEDDING_DIMENSION = 384   # Dimensionality of the Narrative Embedding
BATCH_SIZE = 32
EPOCHS = 75                 # Increased epochs for better convergence of complex model

# Define the target variable
TARGET_COLUMN = 'Emissions Intensity (kg CO₂ per MWh)'
TARGET_CLEANED = 'Emissions_Intensity_kg_CO2_per_MWh'

# Features to use for prediction (Time-Series path input)
FEATURE_COLUMNS = [
    'Revenue (USD)',
    'Net Profit Margin (%)',
    'Energy Efficiency (%)',
    'Renewable Energy Share (%)',
    'Sustainability Score',
    'Innovation Index'
]

# --- Combined Data Preparation and Simulation (Replicating Phase 1 & 2 logic) ---

def generate_augmented_data(file_path):
    """
    Loads, cleans data, and simulates the LLM narratives and embeddings.
    (This function ensures the model script is self-contained and runnable).
    """
    print("--- 1. Data Loading and Narrative Simulation ---")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Please ensure the file is in the correct directory.")
        return None, None

    # --- Robust Column Cleaning Fix (Copied from working Phase 2 logic) ---
    def standardize_column_name(col_name):
        # 1. Start cleaning
        cleaned = col_name
        
        # 2. Aggressively clean up known problematic strings (like CO₂ encoding issues)
        cleaned = cleaned.replace('â\x82\x82', '2') # Handles one common encoding for CO₂
        
        # 3. Replace all non-alphanumeric, non-space characters with an underscore
        cleaned = re.sub(r'[^A-Za-z0-9\s_]', '', cleaned)
        
        # 4. Replace spaces with underscores
        cleaned = cleaned.replace(' ', '_')
        
        # 5. Replace multiple underscores with a single underscore
        cleaned = re.sub(r'_+', '_', cleaned)
        
        # 6. Strip any leading/trailing underscores
        cleaned = cleaned.strip('_')
        
        # FIX: Explicitly handle the target column's known problematic output 
        # to guarantee the required name, as general regex cleaning is failing.
        if 'Emissions_Intensity_kg_CO_per_MWh' in cleaned:
            return TARGET_CLEANED
        
        return cleaned

    # Apply the standardization to all columns
    df.columns = [standardize_column_name(col) for col in df.columns]

    # Convert the dataset into a single pseudo-time-series
    df_ts = df.sort_values(by='Company_ID').reset_index(drop=True)
    
    # List of all clean features and targets needed
    required_cols = ['Company_ID', 'Revenue_USD', 'Net_Profit_Margin', 'Energy_Efficiency', 
                     'Renewable_Energy_Share', 'Sustainability_Score', 'Innovation_Index', 
                     TARGET_CLEANED]
    
    # Filter for the required columns
    # We now use 'df_ts' directly, which has the cleaned columns from the step above.
    df_ts = df_ts[required_cols].dropna()

    # --- End of Cleaning Fix ---

    # Calculate changes for rule-based narrative generation (Phase 2 logic)
    df_ts['Emissions_Change'] = df_ts[TARGET_CLEANED].diff().fillna(0)
    df_ts['Efficiency_Change'] = df_ts['Energy_Efficiency'].diff().fillna(0)
    df_ts['Innovation_Change'] = df_ts['Innovation_Index'].diff().fillna(0)
    
    # Simulate embeddings (Phase 2 logic)
    np.random.seed(42)
    embeddings = np.random.rand(len(df_ts), EMBEDDING_DIMENSION).astype(np.float32)
    
    # FIX: Use efficient concatenation to avoid PerformanceWarning
    embedding_cols = [f'Embedding_{j}' for j in range(EMBEDDING_DIMENSION)]
    embeddings_df = pd.DataFrame(embeddings, columns=embedding_cols, index=df_ts.index)
    df_ts = pd.concat([df_ts, embeddings_df], axis=1)

    df_ts = df_ts.drop(columns=['Emissions_Change', 'Efficiency_Change', 'Innovation_Change', 'Company_ID'])
    
    # Identify feature and embedding columns
    ts_features = [col for col in df_ts.columns if col.startswith(('Revenue', 'Net_Profit', 'Energy_Efficiency', 'Renewable', 'Sustainability_Score', 'Innovation_Index'))]
    # The embedding features start after the TS features
    num_ts_features = len(ts_features)

    # Final data scaling and structuring
    data = df_ts.values
    
    # 1. Scale Features
    scaler_features = MinMaxScaler()
    features_scaled = scaler_features.fit_transform(data[:, :-1]) # All except target
    
    # 2. Scale Target
    scaler_target = MinMaxScaler()
    target_scaled = scaler_target.fit_transform(data[:, -1].reshape(-1, 1))

    # Recombine scaled data
    scaled_data = np.hstack((features_scaled, target_scaled))
    
    return scaled_data, num_ts_features, scaler_target


def create_hybrid_sequences(data, sequence_length, num_ts_features):
    """
    Creates sequences for the dual-input model.
    X_ts: Time-series features (multi-step history)
    X_narrative: Narrative embedding (single step at prediction time)
    y: Target value (single step at prediction time)
    """
    X_ts, X_narrative, y = [], [], []
    
    # Number of total non-target columns (TS Features + Embeddings)
    num_total_features = data.shape[1] - 1 
    
    for i in range(len(data) - sequence_length):
        # A. Time-Series Input (Historical sequence of TS Features only)
        # Select rows [i to i + SEQUENCE_LENGTH - 1] and columns [0 to num_ts_features - 1]
        X_ts.append(data[i: i + sequence_length, :num_ts_features])
        
        # B. Narrative Input (Embedding vector corresponding to the prediction step)
        # Select row [i + SEQUENCE_LENGTH] and columns [num_ts_features to num_total_features - 1]
        X_narrative.append(data[i + sequence_length, num_ts_features:num_total_features])
        
        # C. Target (Target value at the prediction step)
        # Select row [i + SEQUENCE_LENGTH] and the last column
        y.append(data[i + sequence_length, -1])

    return np.array(X_ts), np.array(X_narrative), np.array(y)

# --- 2. Hybrid Fusion Model Definition ---

def build_hybrid_model(ts_input_shape, narrative_input_shape):
    """
    Defines the Dual-Input Fusion Model architecture using Keras Functional API.
    """
    print("\n--- 2. Building Dual-Input Hybrid Fusion Model ---")
    
    # 1. Time-Series Path (Quantitative)
    ts_input = Input(shape=ts_input_shape, name='time_series_input')
    lstm_1 = LSTM(units=64, return_sequences=True)(ts_input)
    dropout_ts_1 = Dropout(0.3)(lstm_1)
    lstm_2 = LSTM(units=32)(dropout_ts_1)
    ts_output = Dense(16, activation='relu', name='ts_feature_vector')(lstm_2)
    
    # 2. Narrative Path (Qualitative)
    narrative_input = Input(shape=narrative_input_shape, name='narrative_embedding_input')
    dense_narrative_1 = Dense(64, activation='relu')(narrative_input)
    dropout_narrative_1 = Dropout(0.3)(dense_narrative_1)
    narrative_output = Dense(16, activation='relu', name='narrative_feature_vector')(dropout_narrative_1)
    
    # 3. Fusion Layer (Concatenation)
    fusion_layer = Concatenate(name='fusion_layer')([ts_output, narrative_output])
    
    # 4. Final Prediction Head
    dense_final_1 = Dense(16, activation='relu')(fusion_layer)
    output = Dense(1, activation='linear', name='emissions_prediction')(dense_final_1)
    
    # Define the final model
    model = Model(inputs=[ts_input, narrative_input], outputs=output, name='Holistic_Horizon_EPM_Hybrid')
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    return model

# --- Main Execution ---

if __name__ == "__main__":
    
    # 1. Prepare and structure data
    scaled_data, num_ts_features, scaler_target = generate_augmented_data(FILE_PATH)
    
    if scaled_data is None:
        exit()

    X_ts, X_narrative, y = create_hybrid_sequences(scaled_data, SEQUENCE_LENGTH, num_ts_features)

    # 2. Split into train and test sets
    test_size = 0.2
    split_index = int(len(X_ts) * (1 - test_size))

    X_ts_train, X_ts_test = X_ts[:split_index], X_ts[split_index:]
    X_narrative_train, X_narrative_test = X_narrative[:split_index], X_narrative[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    print(f"\nTraining Samples: {len(y_train)}, Testing Samples: {len(y_test)}")
    print(f"TS Input Shape (Train): {X_ts_train.shape}")
    print(f"Narrative Input Shape (Train): {X_narrative_train.shape}")

    # 3. Build and train the model
    ts_input_shape = (X_ts_train.shape[1], X_ts_train.shape[2])
    narrative_input_shape = (X_narrative_train.shape[1],)
    
    hybrid_model = build_hybrid_model(ts_input_shape, narrative_input_shape)
    hybrid_model.summary()
    
    print("\n--- 3. Training Hybrid Model ---")
    history = hybrid_model.fit(
        {'time_series_input': X_ts_train, 'narrative_embedding_input': X_narrative_train},
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        verbose=1,
        shuffle=False
    )

    # 4. Evaluate and Inverse Transform
    print("\n--- 4. Evaluating Hybrid Model Performance ---")
    
    # Predict on the test set
    y_pred_scaled = hybrid_model.predict({'time_series_input': X_ts_test, 'narrative_embedding_input': X_narrative_test})
    
    # Inverse transform to get the prediction in the original scale
    y_test_original = scaler_target.inverse_transform(y_test.reshape(-1, 1))
    y_pred_original = scaler_target.inverse_transform(y_pred_scaled)
    
    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(y_pred_original - y_test_original))
    
    print(f"\n--- Holistic Horizon Hybrid Model Final Evaluation ---")
    print(f"Target Variable: {TARGET_COLUMN}")
    print(f"Test MAE (Emissions Intensity): {mae:.2f} kg CO₂ per MWh")
    print("\nNext Steps (Phase 4): Compare this MAE directly to the Phase 1 Baseline to quantify the value of the LLM narrative context.")


--- 1. Data Loading and Narrative Simulation ---

Training Samples: 792, Testing Samples: 198
TS Input Shape (Train): (792, 10, 6)
Narrative Input Shape (Train): (792, 384)

--- 2. Building Dual-Input Hybrid Fusion Model ---


Model: "Holistic_Horizon_EPM_Hybrid"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ time_series_input             │ (None, 10, 6)             │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ (None, 10, 64)            │          18,176 │ time_series_input[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ narrative_embedding_input     │ (None, 384)               │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 10, 64)            │               0 │ lstm_2[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │          24,640 │ narrative_embedding_input… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_3 (LSTM)                 │ (None, 32)                │          12,416 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 64)                │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ ts_feature_vector (Dense)     │ (None, 16)                │             528 │ lstm_3[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ narrative_feature_vector      │ (None, 16)                │           1,040 │ dropout_3[0][0]            │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ fusion_layer (Concatenate)    │ (None, 32)                │               0 │ ts_feature_vector[0][0],   │
│                               │                           │                 │ narrative_feature_vector[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 16)                │             528 │ fusion_layer[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emissions_prediction (Dense)  │ (None, 1)                 │              17 │ dense_2[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 57,345 (224.00 KB)

 Trainable params: 57,345 (224.00 KB)

 Non-trainable params: 0 (0.00 B)


--- 3. Training Hybrid Model ---
Epoch 1/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - loss: 0.1526 - mae: 0.3135 - val_loss: 0.0831 - val_mae: 0.2385
Epoch 2/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0970 - mae: 0.2626 - val_loss: 0.0805 - val_mae: 0.2311
Epoch 3/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 0.0899 - mae: 0.2539 - val_loss: 0.0822 - val_mae: 0.2328
Epoch 4/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - loss: 0.0874 - mae: 0.2532 - val_loss: 0.0797 - val_mae: 0.2333
Epoch 5/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0868 - mae: 0.2527 - val_loss: 0.0795 - val_mae: 0.2342
Epoch 6/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - loss: 0.0843 - mae: 0.2494 - val_loss: 0.0797 - val_mae: 0.2326
Epoch 7/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0843 - mae: 0.2484 - val_loss: 0.0811 - val_mae: 0.2316
Epoch 8/75
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0856 - mae: 0.2500 - val_loss: 0.0777 - val_mae: 0.2309
Epoch 9/75
23/23 ━━━━━━━━━━

In [20]:
import pandas as pd
import numpy as np
import re
import random
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam

# --- Configuration (MUST match previous phases) ---
FILE_PATH = r"C:\Users\acer\Downloads\Enterprise_Sustainable Power Evaluation_Dataset.csv"
SEQUENCE_LENGTH = 10        # Historical steps for Time-Series input
EMBEDDING_DIMENSION = 384   # Dimensionality of the Narrative Embedding
BATCH_SIZE = 32
EPOCHS = 75                 # Consistent training length for fair comparison

# Define the target variable
TARGET_COLUMN = 'Emissions Intensity (kg CO₂ per MWh)'
TARGET_CLEANED = 'Emissions_Intensity_kg_CO2_per_MWh'

# Features to use for prediction (Time-Series path input)
FEATURE_COLUMNS = [
    'Revenue (USD)',
    'Net Profit Margin (%)',
    'Energy Efficiency (%)',
    'Renewable Energy Share (%)',
    'Sustainability Score',
    'Innovation Index'
]

# --- Helper function for robust column cleaning ---
def standardize_column_name_robust(col_name):
    """Aggressively cleans and standardizes a column name."""
    cleaned = col_name
    
    # 1. Handle known encoding/symbol issues (e.g., CO₂ -> CO2)
    cleaned = cleaned.replace('â\x82\x82', '2') 
    
    # 2. Replace all non-alphanumeric, non-space characters with an underscore
    cleaned = re.sub(r'[^A-Za-z0-9\s_]', '_', cleaned)
    
    # 3. Replace spaces with underscores
    cleaned = cleaned.replace(' ', '_')
    
    # 4. Collapse multiple underscores and strip leading/trailing ones
    cleaned = re.sub(r'_+', '_', cleaned).strip('_')
    
    # 5. FIX: Ensure the target column is mapped correctly due to unpredictable source reading
    if 'Emissions_Intensity_kg_CO_per_MWh' in cleaned:
        return TARGET_CLEANED
        
    return cleaned


# --- Data Preparation and Simulation ---

def generate_augmented_data(file_path):
    """
    Loads, cleans, simulates narratives/embeddings, scales data, and returns 
    the necessary components for sequence creation.
    """
    print("--- 1. Data Preparation and Simulation ---")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}.")
        return None, None, None

    # Apply the single, robust standardization function to all columns
    df.columns = [standardize_column_name_robust(col) for col in df.columns]

    # Convert the dataset into a single pseudo-time-series
    df_ts = df.sort_values(by='Company_ID').reset_index(drop=True)

    # The list of columns to use is now based on the standardized names
    clean_feature_cols = [standardize_column_name_robust(col) for col in FEATURE_COLUMNS]
    required_cols = ['Company_ID'] + clean_feature_cols + [TARGET_CLEANED]

    # Filter for the required columns
    # This step will now use the consistently clean column names
    df_ts = df_ts[required_cols].dropna()

    # Identify the feature columns present in the final DataFrame
    ts_features = [col for col in df_ts.columns if col in clean_feature_cols]

    # Calculate changes for narrative simulation (using a generic proxy, as full simulation is complex)
    df_ts['Emissions_Change'] = df_ts[TARGET_CLEANED].diff().fillna(0)
    
    # Simulate embeddings
    np.random.seed(42)
    embeddings = np.random.rand(len(df_ts), EMBEDDING_DIMENSION).astype(np.float32)
    
    # Use efficient concatenation to add embedding columns (avoiding PerformanceWarning)
    embedding_cols = [f'Embedding_{j}' for j in range(EMBEDDING_DIMENSION)]
    embeddings_df = pd.DataFrame(embeddings, columns=embedding_cols, index=df_ts.index)
    df_ts = pd.concat([df_ts, embeddings_df], axis=1)

    # Drop non-feature columns
    df_ts = df_ts.drop(columns=[col for col in df_ts.columns if 'Change' in col] + ['Company_ID'])
    
    # Final data scaling and structuring
    data = df_ts.values
    num_ts_features = len(ts_features)
    
    # 1. Scale Features (all except target)
    scaler_features = MinMaxScaler()
    features_scaled = scaler_features.fit_transform(data[:, :-1])
    
    # 2. Scale Target
    scaler_target = MinMaxScaler()
    target_scaled = scaler_target.fit_transform(data[:, -1].reshape(-1, 1))

    # Recombine scaled data
    scaled_data = np.hstack((features_scaled, target_scaled))
    
    return scaled_data, num_ts_features, scaler_target


def create_sequences(data, sequence_length, num_ts_features):
    """
    Creates sequences for the dual-input model (Hybrid) and single-input (Baseline).
    Returns X_ts, X_narrative, y.
    """
    X_ts, X_narrative, y = [], [], []
    num_total_features = data.shape[1] - 1 
    
    for i in range(len(data) - sequence_length):
        # A. Time-Series Input (Historical sequence of TS Features only)
        # Columns [0 to num_ts_features - 1]
        X_ts.append(data[i: i + sequence_length, :num_ts_features])
        
        # B. Narrative Input (Embedding vector at prediction step)
        # Columns [num_ts_features to num_total_features - 1]
        X_narrative.append(data[i + sequence_length, num_ts_features:num_total_features])
        
        # C. Target (Target value at prediction step)
        y.append(data[i + sequence_length, -1])

    return np.array(X_ts, dtype=np.float32), np.array(X_narrative, dtype=np.float32), np.array(y, dtype=np.float32)

# --- TensorFlow Dataset Creation ---

def create_tf_datasets(X_ts_train, X_ts_val, X_ts_test, X_narrative_train, X_narrative_val, X_narrative_test, y_train, y_val, y_test, batch_size):
    """Creates optimized TensorFlow Dataset objects for training and testing, incorporating validation set."""
    
    # Baseline Dataset (Single Input: Time-Series only)
    ds_baseline_train = tf.data.Dataset.from_tensor_slices((X_ts_train, y_train))
    ds_baseline_val = tf.data.Dataset.from_tensor_slices((X_ts_val, y_val))
    ds_baseline_test = tf.data.Dataset.from_tensor_slices((X_ts_test, y_test))
    
    # Hybrid Dataset (Dual Input: Time-Series and Narrative)
    X_train_hybrid = {'time_series_input': X_ts_train, 'narrative_embedding_input': X_narrative_train}
    X_val_hybrid = {'time_series_input': X_ts_val, 'narrative_embedding_input': X_narrative_val}
    X_test_hybrid = {'time_series_input': X_ts_test, 'narrative_embedding_input': X_narrative_test}
    
    ds_hybrid_train = tf.data.Dataset.from_tensor_slices((X_train_hybrid, y_train))
    ds_hybrid_val = tf.data.Dataset.from_tensor_slices((X_val_hybrid, y_val))
    ds_hybrid_test = tf.data.Dataset.from_tensor_slices((X_test_hybrid, y_test))

    # Apply batching and prefetching for performance
    ds_baseline_train = ds_baseline_train.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_baseline_val = ds_baseline_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_baseline_test = ds_baseline_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    ds_hybrid_train = ds_hybrid_train.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_hybrid_val = ds_hybrid_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_hybrid_test = ds_hybrid_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return (ds_baseline_train, ds_baseline_val, ds_baseline_test), (ds_hybrid_train, ds_hybrid_val, ds_hybrid_test)


# --- Model Definitions ---

def build_baseline_model(ts_input_shape):
    """LSTM Baseline Model (Phase 1)."""
    model = Sequential([
        Input(shape=ts_input_shape, name='ts_baseline_input'), # Use Input layer to avoid warning
        LSTM(units=50, return_sequences=True, name='lstm_baseline_1'),
        Dropout(0.2),
        LSTM(units=50, return_sequences=False, name='lstm_baseline_2'),
        Dropout(0.2),
        Dense(units=1, activation='linear', name='baseline_output')
    ], name="EPM_LSTM_Baseline")
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def build_hybrid_model(ts_input_shape, narrative_input_shape):
    """Hybrid Fusion Model (Phase 3)."""
    # 1. Time-Series Path (Quantitative)
    ts_input = Input(shape=ts_input_shape, name='time_series_input')
    lstm_1 = LSTM(units=64, return_sequences=True)(ts_input)
    dropout_ts_1 = Dropout(0.3)(lstm_1)
    lstm_2 = LSTM(units=32)(dropout_ts_1)
    ts_output = Dense(16, activation='relu', name='ts_feature_vector')(lstm_2)
    
    # 2. Narrative Path (Qualitative)
    narrative_input = Input(shape=narrative_input_shape, name='narrative_embedding_input')
    dense_narrative_1 = Dense(64, activation='relu')(narrative_input)
    dropout_narrative_1 = Dropout(0.3)(dense_narrative_1)
    narrative_output = Dense(16, activation='relu', name='narrative_feature_vector')(dropout_narrative_1)
    
    # 3. Fusion Layer
    fusion_layer = Concatenate(name='fusion_layer')([ts_output, narrative_output])
    
    # 4. Final Prediction Head
    dense_final_1 = Dense(16, activation='relu')(fusion_layer)
    output = Dense(1, activation='linear', name='emissions_prediction')(dense_final_1)
    
    model = Model(inputs=[ts_input, narrative_input], outputs=output, name='Holistic_Horizon_EPM_Hybrid')
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    return model

# --- Evaluation Function ---

def evaluate_model(model, ds_test, y_test_original_scale, scaler_target, model_name):
    """Evaluates the model using the TensorFlow Dataset."""
    
    # Predict
    y_pred_scaled = model.predict(ds_test, verbose=0)
    
    # Inverse transform
    y_pred_original = scaler_target.inverse_transform(y_pred_scaled)
    
    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(y_pred_original - y_test_original_scale))
    
    print(f"\n--- Evaluation: {model_name} ---")
    print(f"Test MAE ({TARGET_COLUMN}): {mae:.4f} kg CO₂ per MWh")
    return mae


# --- Main Execution ---

if __name__ == "__main__":
    
    # 1. Prepare and structure data
    scaled_data, num_ts_features, scaler_target = generate_augmented_data(FILE_PATH)
    if scaled_data is None:
        exit()

    X_ts, X_narrative, y = create_sequences(scaled_data, SEQUENCE_LENGTH, num_ts_features)

    # 2. Split into train, validation, and test sets (consistent split)
    test_size_ratio = 0.2
    val_size_ratio = 0.1 # 10% of the training data
    
    # Split into Train + Val and Test
    split_index = int(len(X_ts) * (1 - test_size_ratio))
    X_ts_tv, X_ts_test = X_ts[:split_index], X_ts[split_index:]
    X_narrative_tv, X_narrative_test = X_narrative[:split_index], X_narrative[split_index:]
    y_tv, y_test = y[:split_index], y[split_index:]

    # Split Train + Val into Train and Val
    val_index = int(len(X_ts_tv) * (1 - val_size_ratio))
    X_ts_train, X_ts_val = X_ts_tv[:val_index], X_ts_tv[val_index:]
    X_narrative_train, X_narrative_val = X_narrative_tv[:val_index], X_narrative_tv[val_index:]
    y_train, y_val = y_tv[:val_index], y_tv[val_index:]
    
    # Get the test target values in the original scale for final evaluation 
    y_test_original_scale = scaler_target.inverse_transform(y_test.reshape(-1, 1))

    print(f"\nTraining Samples: {len(y_train)}, Validation Samples: {len(y_val)}, Testing Samples: {len(y_test)}")
    
    ts_input_shape = (X_ts_train.shape[1], X_ts_train.shape[2])
    narrative_input_shape = (X_narrative_train.shape[1],)
    
    # 3. Create TensorFlow Datasets (Optimization step)
    (ds_baseline_train, ds_baseline_val, ds_baseline_test), (ds_hybrid_train, ds_hybrid_val, ds_hybrid_test) = \
        create_tf_datasets(X_ts_train, X_ts_val, X_ts_test, X_narrative_train, X_narrative_val, X_narrative_test, y_train, y_val, y_test, BATCH_SIZE)
    
    # --- 4. Run Baseline Model ---
    
    baseline_model = build_baseline_model(ts_input_shape)
    print("\n--- Training LSTM Baseline Model ---")
    baseline_model.fit(
        ds_baseline_train,
        epochs=EPOCHS,
        validation_data=ds_baseline_val,
        verbose=0,
    )
    mae_baseline = evaluate_model(baseline_model, ds_baseline_test, y_test_original_scale, scaler_target, "LSTM Baseline Model")
    
    # --- 5. Run Hybrid Model ---

    hybrid_model = build_hybrid_model(ts_input_shape, narrative_input_shape)
    print("\n--- Training Holistic Horizon Hybrid Model ---")
    hybrid_model.fit(
        ds_hybrid_train,
        epochs=EPOCHS,
        validation_data=ds_hybrid_val,
        verbose=0,
    )
    mae_hybrid = evaluate_model(hybrid_model, ds_hybrid_test, y_test_original_scale, scaler_target, "Hybrid Fusion Model")
    
    # --- 6. Comparative Analysis ---
    
    print("\n" + "="*50)
    print("      Holistic Horizon EPM Project: Final Comparison")
    print("="*50)
    print(f"Target Metric: {TARGET_COLUMN}")
    print(f"1. Pure Time-Series (LSTM Baseline) MAE: {mae_baseline:.4f}")
    print(f"2. Multi-Modal Hybrid (Fusion) MAE:       {mae_hybrid:.4f}")
    
    if mae_hybrid < mae_baseline:
        improvement = ((mae_baseline - mae_hybrid) / mae_baseline) * 100
        print(f"\nConclusion: Hybrid model outperformed the Baseline by {improvement:.2f}%.")
        print("This suggests that the **qualitative narrative context** (simulated LLM embeddings) adds significant predictive power and reduces forecast error.")
    elif mae_hybrid > mae_baseline:
        decline = ((mae_hybrid - mae_baseline) / mae_baseline) * 100
        print(f"\nConclusion: Hybrid model underperformed the Baseline by {decline:.2f}%.")
        print("This suggests that the simulated narrative context might be introducing noise or that the current fusion architecture needs tuning.")
    else:
        print("\nConclusion: The models performed identically. Further tuning is required.")
    
    print("="*50)


--- 1. Data Preparation and Simulation ---

Training Samples: 712, Validation Samples: 80, Testing Samples: 198

--- Training LSTM Baseline Model ---

--- Evaluation: LSTM Baseline Model ---
Test MAE (Emissions Intensity (kg CO₂ per MWh)): 0.2513 kg CO₂ per MWh

--- Training Holistic Horizon Hybrid Model ---

--- Evaluation: Hybrid Fusion Model ---
Test MAE (Emissions Intensity (kg CO₂ per MWh)): 0.2964 kg CO₂ per MWh

      Holistic Horizon EPM Project: Final Comparison
Target Metric: Emissions Intensity (kg CO₂ per MWh)
1. Pure Time-Series (LSTM Baseline) MAE: 0.2513
2. Multi-Modal Hybrid (Fusion) MAE:       0.2964

Conclusion: Hybrid model underperformed the Baseline by 17.96%.
This suggests that the simulated narrative context might be introducing noise or that the current fusion architecture needs tuning.


In [21]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import re

# --- Configuration (Matching the EPM project structure) ---
# NOTE: Using the generic filename for platform compatibility
FILE_PATH = r"C:\Users\acer\Downloads\Enterprise_Sustainable Power Evaluation_Dataset.csv"
TARGET_COLUMN = 'Emissions Intensity (kg CO₂ per MWh)'
TARGET_CLEANED = 'Emissions_Intensity_kg_CO2_per_MWh'

# --- Robust Column Cleaning Function (Replicating ML analysis success) ---
@st.cache_data
def standardize_column_name_robust(col_name):
    """Aggressively cleans and standardizes a column name."""
    cleaned = col_name
    
    # 1. Handle known encoding/symbol issues (e.g., CO₂ -> CO2)
    cleaned = cleaned.replace('â\x82\x82', '2') 
    
    # 2. Replace all non-alphanumeric, non-space characters with an underscore
    cleaned = re.sub(r'[^A-Za-z0-9\s_]', '', cleaned)
    
    # 3. Replace spaces with underscores
    cleaned = cleaned.replace(' ', '_')
    
    # 4. Collapse multiple underscores and strip leading/trailing ones
    cleaned = re.sub(r'_+', '_', cleaned).strip('_')
    
    # 5. FIX: Ensure the target column is mapped correctly
    if 'Emissions_Intensity_kg_CO_per_MWh' in cleaned:
        return TARGET_CLEANED
        
    return cleaned

# --- Data Loading and Cleaning ---
@st.cache_data
def load_and_preprocess_data(file_path):
    """Loads, cleans, and prepares the dataset."""
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        st.error(f"Error: File not found at {file_path}. Please ensure the CSV is accessible.")
        return pd.DataFrame()

    # Apply the single, robust standardization function to all columns
    df.columns = [standardize_column_name_robust(col) for col in df.columns]

    # Convert to pseudo-time-series by sorting
    df_ts = df.sort_values(by='Company_ID').reset_index(drop=True)
    
    # Create a simple time index for plotting the simulated time-series
    df_ts['Time_Step'] = df_ts.index
    
    # Ensure key columns are float for plotting
    key_cols = [TARGET_CLEANED, 'Renewable_Energy_Share', 'Sustainability_Score', 'Net_Profit_Margin']
    for col in key_cols:
        if col in df_ts.columns:
            df_ts[col] = pd.to_numeric(df_ts[col], errors='coerce')
            
    df_ts = df_ts.dropna()
    return df_ts

df = load_and_preprocess_data(FILE_PATH)

# Check if data loaded successfully
if df.empty:
    st.stop()

# --- Dashboard Layout and Styling ---
st.set_page_config(
    layout="wide", 
    page_title="Holistic Horizon EPM Prediction Dashboard", 
    initial_sidebar_state="collapsed"
)

# Apply custom CSS for dark mode look and clean typography
st.markdown("""
<style>
    .stApp {
        background-color: #0d1117; /* Dark background */
        color: #c9d1d9; /* Light text */
    }
    .stPlotly, .stAlert {
        border-radius: 8px;
        padding: 10px;
        background-color: #161b22; /* Slightly lighter container for contrast */
    }
    h1, h2, h3 {
        color: #58a6ff; /* Blue for headings */
    }
    .st-cd, .st-ce {
        background-color: #161b22;
        border-radius: 8px;
        padding: 10px;
    }
    .st-emotion-cache-1629p8f { /* Targetting metrics container for better alignment */
        gap: 1.5rem;
    }
</style>
""", unsafe_allow_html=True)


st.title("🌌 Holistic Horizon EPM Dashboard: Multi-Modal Integrated Prediction")
st.markdown("### Fusing Quantitative Time-Series Data with Qualitative LLM Context")

st.info("""
    This dashboard provides a visualization of the Enterprise Performance Management ($\text{EPM}$) metrics used for training the Hybrid Fusion Model. The core objective is to reduce prediction error for **Emissions Intensity** by incorporating **simulated narrative context** ($\text{LLM}$ embeddings).
""")

# --- 1. KPI Overview (Gauges and Metrics) ---
st.markdown("---")
st.subheader("Key Performance Indicators (EPM Snapshot)")

if TARGET_CLEANED in df.columns:
    avg_emissions = df[TARGET_CLEANED].mean()
    avg_sustainability = df['Sustainability_Score'].mean()
    avg_renewable = df['Renewable_Energy_Share'].mean()
    avg_profit = df['Net_Profit_Margin'].mean()

    col1, col2, col3, col4 = st.columns(4)

    # Metric 1: Emissions (Goal: Lower)
    col1.metric("Avg. Emissions Intensity", f"{avg_emissions:.2f} kg/MWh", delta_color="inverse")

    # Metric 2: Net Profit Margin
    col2.metric("Avg. Net Profit Margin", f"{avg_profit:.2f} %", delta=f"{df['Net_Profit_Margin'].std():.2f} Std Dev")

    # Metric 3: Sustainability Score (Gauge)
    with col3:
        fig_sustainability = go.Figure(go.Indicator(
            mode = "gauge+number",
            value = avg_sustainability,
            title = {'text': "Avg. Sustainability Score", 'font': {'size': 14}},
            gauge = {
                'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkgray"},
                'bar': {'color': "#58a6ff"},
                'steps': [
                    {'range': [0, 60], 'color': "red"},
                    {'range': [60, 80], 'color': "yellow"},
                    {'range': [80, 100], 'color': "green"}],
                'threshold': {'line': {'color': "white", 'width': 4}, 'thickness': 0.75, 'value': 85}}
        ))
        fig_sustainability.update_layout(height=200, margin=dict(t=50, b=0, l=10, r=10), template="plotly_dark")
        st.plotly_chart(fig_sustainability, use_container_width=True)

    # Metric 4: Renewable Share (Gauge)
    with col4:
        fig_renewable = go.Figure(go.Indicator(
            mode = "gauge+number",
            value = avg_renewable,
            title = {'text': "Avg. Renewable Share", 'font': {'size': 14}},
            gauge = {
                'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkgray"},
                'bar': {'color': "#2A803B"}, # Darker green for energy
                'steps': [
                    {'range': [0, 25], 'color': "red"},
                    {'range': [25, 50], 'color': "yellow"},
                    {'range': [50, 100], 'color': "green"}],
                'threshold': {'line': {'color': "white", 'width': 4}, 'thickness': 0.75, 'value': 60}}
        ))
        fig_renewable.update_layout(height=200, margin=dict(t=50, b=0, l=10, r=10), template="plotly_dark")
        st.plotly_chart(fig_renewable, use_container_width=True)


# --- 2. Dual-Axis Time-Series Trend ---
st.markdown("---")
st.subheader("Quantitative Path Visualization: Simulated EPM Trend")

tab1, tab2 = st.tabs(["Dual-Axis Trend (Target vs. Driver)", "Feature Distributions"])

with tab1:
    fig_ts = go.Figure()

    # Emissions (Primary Axis) - Orange/Red for warning/emissions
    fig_ts.add_trace(go.Scatter(
        x=df['Time_Step'], 
        y=df[TARGET_CLEANED], 
        mode='lines', 
        name=TARGET_COLUMN, 
        yaxis='y1',
        line=dict(color='#ff7f0e', width=3)
    ))

    # Renewable Energy Share (Secondary Axis) - Blue/Green for progress
    fig_ts.add_trace(go.Scatter(
        x=df['Time_Step'], 
        y=df['Renewable_Energy_Share'], 
        mode='lines', 
        name='Renewable Energy Share (%)', 
        yaxis='y2',
        line=dict(color='#1f77b4', dash='dash', width=2)
    ))

    fig_ts.update_layout(
        title='Simulated EPM Trend: Emissions Intensity (Target) vs. Renewable Share (Feature)',
        xaxis_title='Simulated Time Step (Company Index)',
        yaxis=dict(
            title=TARGET_COLUMN,
            titlefont=dict(color='#ff7f0e'),
            tickfont=dict(color='#ff7f0e'),
            gridcolor='#161b22'
        ),
        yaxis2=dict(
            title='Renewable Energy Share (%)',
            titlefont=dict(color='#1f77b4'),
            tickfont=dict(color='#1f77b4'),
            overlaying='y',
            side='right',
            gridcolor='#161b22'
        ),
        height=550,
        template="plotly_dark"
    )
    st.plotly_chart(fig_ts, use_container_width=True)
    st.markdown("""
        *Observation:* This dual-axis chart shows the sequential data fed to the $\text{LSTM}$ model. The **Hybrid Model** uses this history *plus* the **Narrative Embedding** corresponding to the prediction step to capture non-linear market/policy impacts.
    """)

with tab2:
    selected_feature = st.selectbox(
        'Select a Feature to view its Distribution:',
        options=[
            'Revenue_USD', 'Net_Profit_Margin', 'Energy_Efficiency', 
            'Sustainability_Score', TARGET_CLEANED
        ]
    )
    
    if selected_feature in df.columns:
        fig_dist = px.histogram(
            df, 
            x=selected_feature, 
            title=f'Distribution of {selected_feature.replace("_", " ")}',
            color_discrete_sequence=['#5D9C3E'],
            template="plotly_dark"
        )
        fig_dist.update_layout(height=450)
        st.plotly_chart(fig_dist, use_container_width=True)

# --- 3. Feature Relationship (Scatter Plot) ---
st.markdown("---")
st.subheader("Feature Correlation: Sustainability vs. Emissions")

if 'Sustainability_Score' in df.columns:
    fig_scatter = px.scatter(
        df, 
        x='Sustainability_Score', 
        y=TARGET_CLEANED, 
        color='Net_Profit_Margin', 
        size='Revenue_USD', # Use Revenue to denote company size/impact
        hover_data=['Company_ID'],
        title=f'Emissions Intensity vs. Sustainability Score, Colored by Profit Margin',
        labels={
            TARGET_CLEANED: TARGET_COLUMN,
            'Sustainability_Score': 'Overall Sustainability Score (0-100)',
            'Net_Profit_Margin': 'Net Profit Margin (%)'
        },
        color_continuous_scale=px.colors.sequential.Viridis,
        template='plotly_dark'
    )
    fig_scatter.update_layout(height=600, coloraxis_colorbar=dict(title="Profit Margin %"))
    st.plotly_chart(fig_scatter, use_container_width=True)
    st.markdown("""
        *Insight:* Outliers in this plot—companies with high scores but high emissions, or low scores but high profit—are where the **Narrative Path** is most crucial. The qualitative context can explain these non-linear relationships, which a pure time-series model would struggle to capture.
    """)


# --- 4. Project Interpretation and Architecture ---
st.markdown("---")
st.subheader("Hybrid Model Architecture: Fusing Whispers to Roars")

st.markdown("""
    The "Holistic Horizon" model is built on a **Dual-Input Fusion Architecture** to achieve superior $\text{EPM}$ prediction:

    1.  **Quantitative Path (Time-Series $\text{LSTM}$):** Learns the temporal dependencies and patterns inherent in the numerical $\text{KPIs}$ (e.g., historical revenue, efficiency, and emissions).
    2.  **Qualitative Path ($\text{Dense Network}$):** Processes the **Narrative Embedding Vector** (simulated $\text{LLM}$ output) to capture the semantic context, such as policy shifts, strategic management decisions, or unplanned operational events.
    3.  **Fusion:** The feature vectors from both paths are **concatenated** at a bottleneck layer, allowing the model to learn combined weights and predict the target based on **both historical trends and qualitative context**.

    This integration is why the Hybrid Model is expected to outperform the pure $\text{LSTM}$ baseline.
""")

st.code(
    """
    # Conceptual Keras Fusion
    ts_input = Input(shape=(SEQUENCE_LENGTH, num_ts_features))
    narrative_input = Input(shape=(EMBEDDING_DIMENSION,))

    ts_output = LSTM_path(ts_input)            # Quantitative feature vector (e.g., 16 units)
    narrative_output = Dense_path(narrative_input) # Qualitative feature vector (e.g., 16 units)

    fusion = Concatenate()([ts_output, narrative_output])
    prediction = Dense(1)(fusion)
    """
)


2025-10-07 11:14:22.478 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-07 11:14:22.482 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-07 11:14:23.033 
  command:

    streamlit run C:\Users\acer\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-07 11:14:23.038 No runtime found, using MemoryCacheStorageManager
2025-10-07 11:14:23.078 No runtime found, using MemoryCacheStorageManager
2025-10-07 11:14:25.104 Session state does not function when running a script without `streamlit run`


DeltaGenerator()